# 此筆記介紹 many to many這個種類的 RNN。

所謂many to many即：

於多個(many)連續的時間點讀取資訊(例如：$\vec{x}_{t=0},\vec{x}_{t=1},\vec{x}_{t=2},...,\vec{x}_{t=n}$)，然後輸出於各相應時間點(many)的預測結果(例如：$\vec{out}_{t=0},\vec{out}_{t=1},\vec{out}_{t=2},...,\vec{out}_{t=n}$)。

---

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import numpy as np

import pandas as pd

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense

## 拿$\cos(t_1)$, $\cos(t_2)$, ..., $\cos(t_n)$去預測$\sin(t_1)$, $\sin(t_2)$, ..., $\sin(t_n)$

### 1. 畫出$\sin(x)$和$\cos(x)$

In [ ]:
line=np.linspace(0, 2.*np.pi, 500)
in_x=np.cos(2.*np.pi*line)
in_y=np.sin(2.*np.pi*line)

plt.scatter(line,in_x)
plt.scatter(line,in_y)

### 2. 產生用於訓練和測試的資料。

In [ ]:
line = np.linspace(0, 2.*np.pi, 500)
in_x = np.cos(2.*np.pi*line)
in_y = np.sin(2.*np.pi*line)

n = 20 # 每個樣本有20個用於訓練的x
m = n  # 每個樣本有20個用於訓練的y
num_samples = 200 # 200個樣本

# 建立訓練用樣本 train_x, train_y
train_x = np.zeros((num_samples,n),
                   dtype=np.float32 )
train_y = np.zeros((num_samples,m),
                   dtype=np.float32 )
rand_nums = np.zeros(num_samples,
                     dtype=np.int32)

for j in range(num_samples):
    rand_nums[j] = np.random.choice(500-n)
    train_x[j,:] = in_x[rand_nums[j]:rand_nums[j]+n]
    train_y[j,:] = in_y[rand_nums[j]:rand_nums[j]+m]

# 繪製其中一個樣本的x和y
rand_choice = 10
plt.plot(line[rand_nums[rand_choice]:rand_nums[rand_choice]+n],
         train_x[rand_choice,:], ms=7, marker='o', label="train_x")
plt.plot(line[rand_nums[rand_choice]:rand_nums[rand_choice]+n],
         train_y[rand_choice,:], ms=7, marker='o', label="train_y")
plt.legend()
plt.show()

train_x = train_x.reshape(*train_x.shape, 1)
train_y = train_y.reshape(*train_y.shape, 1)

### 3. 建立並訓練模型，將模型的訓練情形畫出。

In [ ]:
from tensorflow.keras.layers import TimeDistributed

In [ ]:
hidden_neurons = 30

time_dim = 20
seq_dim = 1

model = Sequential()
model.add(SimpleRNN(input_shape=(time_dim,seq_dim),
                    units=hidden_neurons,
                    return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='relu')))
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse'])
model.summary()

history = model.fit(train_x,train_y,
                    epochs=100, batch_size=32, validation_split=0.2)

plt.yscale('log')
plt.plot(history.history['mean_squared_error'], ms=5, marker='o', label='mse')
plt.plot(history.history['val_mean_squared_error'], ms=5, marker='o', label='val mse')
plt.legend()
plt.show()

### 4. 建立新模型：將一個SimpleRNN層改成兩個，訓練並畫出訓練結果。

In [ ]:
hidden_neurons = 100

time_dim = 20
seq_dim = 1

model = Sequential()
model.add(SimpleRNN(input_shape=(time_dim,seq_dim),
                    units=hidden_neurons,
                    return_sequences=True))
model.add(SimpleRNN(units=hidden_neurons,
                    return_sequences=True))
model.add(TimeDistributed(Dense(1,activation='tanh')))
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mse'])
model.summary()

history=model.fit(train_x,train_y,
                  epochs=100, batch_size=20, validation_split=0.2)

plt.yscale('log')
plt.plot(history.history['mean_squared_error'], ms=5, marker='o', label='mse')
plt.plot(history.history['val_mean_squared_error'], ms=5, marker='o', label='val mse')
plt.legend()
plt.show()

兩層RNN可得到更佳模型。

### 5. 隨機丟一個樣本進去做預測。

In [ ]:
phi = np.pi/3.
line = np.linspace(0,2.*np.pi,500)
in_x = np.cos(2.*np.pi*line+phi)
in_y = np.sin(2.*np.pi*line+phi)

n = 20
m = n

test_x = np.zeros(n, dtype=np.float32)
test_y = np.zeros(m, dtype=np.float32)
rand_num = np.random.choice(500-n)

test_x = in_x[rand_num:rand_num+n]
test_y = in_y[rand_num:rand_num+m]

test_x = test_x.reshape(1,test_x.shape[0],1)

y_pred = model.predict( test_x )
y_pred = y_pred.reshape(-1)

plt.scatter(x=line[rand_num:rand_num+n],
            y=in_x[rand_num:rand_num+n], label="x_in")
plt.scatter(x=line[rand_num:rand_num+n],
            y=in_y[rand_num:rand_num+n], label="y")
plt.scatter(x=line[rand_num:rand_num+m],
            y=y_pred, label="y_pred")
plt.legend()
plt.show()

只有一個點的時候，不太可能做出好的預測。但有兩三個點的時候，機器慢慢就知道這是什麼樣趨勢的曲線，然後須預測出什麼樣的曲線。